In [5]:
import numpy as np 
from scipy.io import loadmat
import pandas as pd

/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [6]:
picnames=[e.strip() for e in open('namelist.txt', 'r').readlines()]
query_names=picnames[:55]
print(query_names)

['hertford_2', 'cornmarket_5', 'christ_church_5', 'balliol_4', 'ashmolean_3', 'bodleian_1', 'ashmolean_4', 'all_souls_2', 'hertford_5', 'magdalen_5', 'all_souls_1', 'keble_1', 'ashmolean_5', 'bodleian_5', 'magdalen_4', 'hertford_3', 'pitt_rivers_2', 'radcliffe_camera_1', 'pitt_rivers_4', 'balliol_5', 'christ_church_2', 'radcliffe_camera_5', 'all_souls_5', 'all_souls_4', 'balliol_3', 'keble_4', 'cornmarket_1', 'cornmarket_3', 'pitt_rivers_1', 'pitt_rivers_3', 'keble_2', 'keble_5', 'ashmolean_2', 'ashmolean_1', 'magdalen_2', 'bodleian_4', 'magdalen_1', 'bodleian_2', 'magdalen_3', 'balliol_2', 'cornmarket_4', 'radcliffe_camera_3', 'radcliffe_camera_4', 'bodleian_3', 'christ_church_3', 'keble_3', 'balliol_1', 'christ_church_4', 'radcliffe_camera_2', 'cornmarket_2', 'hertford_1', 'all_souls_3', 'christ_church_1', 'pitt_rivers_5', 'hertford_4']


In [7]:
features=loadmat("Oxford-COT2.mat")

#查看有返回的类型和他的键
print("keys:",features.keys())
#分离querys和features
features = np.asarray(features['our_feature'])
querys=features[:55,:]
features,features_number = features[55:,:-1],features[55:,-1] 
querys,querys_number = querys[:,:-1],querys[:,-1] 
print(querys_number)
#得到一范数度量
result=np.sum(np.abs(features-querys[:,np.newaxis,:]),axis=2)
# result=np.power(np.sum(np.power(features-querys[:,np.newaxis,:],2),axis=2),0.5)
rank_ID = np.argsort(result,axis=1) + 55
print(rank_ID.shape)
print(rank_ID)

keys: dict_keys(['__header__', 'our_feature', '__globals__', '__version__'])
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54.]
(55, 5063)
[[1421 4036 5008 ... 4175 3468 1510]
 [1572  250   75 ... 3660  265  569]
 [ 750 2357 1008 ... 1414  265  569]
 ...
 [4990 2017 5070 ... 1414 3612  569]
 [1245  464 3286 ... 1510  265  569]
 [4677 2010 4792 ... 4175 4952 1510]]


In [9]:
with open("result.txt",'w') as output:
    for i,query_num in enumerate(querys_number):
        query_name =  query_names[int(query_num)]
        line = [query_name]
        picnames=np.asarray(picnames)
        for img_name in picnames[rank_ID[i,:]]:
            line.append(img_name)
        output.write(' '.join(line) + '\n')

In [10]:
from compute import load_list,compute_ap
def load_result(fname):
    result={}
    for e in open(fname,'r').readlines():
        line=e.strip().split(' ')
        result[line[0]]=line[1:]
    return result
result=load_result('result.txt')
gt_path='/share/dong/Data/OxfordBuilding/gtfiles/'
ap={}
for query,ranked_list in result.items():
    querypath=gt_path+query
    pos_set = list(set(load_list("%s_good.txt" % querypath) + load_list("%s_ok.txt" % querypath)))
    junk_set = load_list("%s_junk.txt" % querypath)
    ap[query]=compute_ap(pos_set, junk_set, ranked_list)
aplist = ap.values()
print(ap)
print(sum(aplist)/len(aplist))
print(len(aplist))

{'bodleian_3': 0.023126226280771354, 'radcliffe_camera_1': 0.1729164408702777, 'christ_church_2': 0.02628239493140978, 'ashmolean_3': 0.05227912184208781, 'all_souls_4': 0.029162931425600336, 'keble_5': 0.14455270004903556, 'ashmolean_1': 0.011027302997705614, 'all_souls_3': 0.05536825394308954, 'bodleian_5': 0.008611388455993409, 'keble_3': 0.14590507415319603, 'radcliffe_camera_2': 0.18492813983670697, 'all_souls_1': 0.04185720592237557, 'pitt_rivers_1': 0.16903165520846428, 'pitt_rivers_3': 0.1700627984263415, 'pitt_rivers_5': 0.16957010177953974, 'ashmolean_5': 0.006393431988048779, 'hertford_4': 0.06149420437161867, 'keble_1': 0.14628243385409104, 'christ_church_5': 0.022466800764361028, 'keble_4': 0.03786419301094096, 'balliol_3': 0.09946900255248983, 'balliol_2': 0.08687538284939829, 'cornmarket_1': 0.0026456734037806256, 'hertford_1': 0.03718687053486685, 'keble_2': 0.14475164518828207, 'bodleian_4': 0.007436472335165349, 'balliol_5': 0.0882702335858389, 'balliol_1': 0.09447825